##Globals in a dask graph

In [2]:
x = 15

def f():
    global x
    x = x + 1
    return x

def inc(x):
    return x + 1

def add(x, y):
    return x + y

In [4]:
import dask.threaded
import dask.multiprocessing

dsk = {"1": (f,),
       "2": (add, "1", "1"),
       "3": (add, "1", 0),
       "4": (add, "1", 0),
       "5": (add, "3", "4")}


###Questions
1. What will dask.threading.get(dsk, "2") return?
  * 30?
  * 31?
  * Some other value?
  * Failure?
2. What if dask.threading.get(dsk, "2") is called again?
  * 32?
  * 33?
  * Some other value?
  * Failure?
3. What will dask.multiprocessing.get(dsk, "2") return?
  * 34?
  * 35?
  * Some other value?
  * Failure?
4. What if dask.multiprocessing.get(dsk, "2") is called again?
  * 36?
  * 37?
  * Some other value?
  * Failure?

####Moral of the story
* Globals are always fraught with peril.

##Classes in a dask graph

In [5]:
class Foo(object):
    def __init__(self, arg1):
        self.arg1 = arg1
        
    #Coming to you live from the JVM!!
    #<drumroll>
    #Getters and Setters!!!! WOOOOOOO!!!!
    def get(self):
        return self.arg1
    
    def set(self, arg):
        self.arg1 = arg
        


In [6]:
dsk = {}

###Questions
* Should classes be avoided in dask graphs?
  * Maybe
* 

##Generators in a dask graph

In [10]:
import operator

def gen1(x):
    for i in xrange(x):
        yield -i
        

dsk = {"gen1": gen1(100),
       "x":(operator.methodcaller('next'),"gen1")}

print dask.threaded.get(dsk, 'x')
print dask.threaded.get(dsk, 'x')
print dask.threaded.get(dsk, 'x')
print dask.threaded.get(dsk, 'x')
print dask.threaded.get(dsk, 'x')

0
-1
-2
-3
-4


###Questions
* Why would you want to do such a thing?

##Keys in the dask graph


In [11]:
dsk = {"1": 100,
       "2": 200,
        3: 300,
       "4": (add, "1", "2"),
       "5": (add, 2, 3)}

###Questions
 * What will dask.threaded.get(dsk, "4") return?
 * What will dask.threaded.get(dsk, "5") return?

In [12]:
print dask.threaded.get(dsk, "4")
print dask.threaded.get(dsk, "5")

300
302
